In [1]:
import xarray as xr

/opt/conda/envs/pyAODN/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/opt/conda/envs/pyAODN/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(scheduler_port=8789)
# cluster

In [3]:
from distributed import LocalCluster
cluster = LocalCluster(n_workers=3,processes=3,threads_per_worker=1,)
cluster

In [4]:
from distributed import Client
client=Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:44195 Dashboard: /proxy/8787/status,Cluster Workers: 3 Cores: 3 Memory: 3.09 GB


In [18]:
import glob
files=glob.glob('/home/jovyan/shared-data/aqua_netcdf/*2014*')
files=sorted(files)
print(len(files))

365


And open all the files as a multi-file dataset with XArray

In [19]:
ds = xr.open_mfdataset(files,chunks={'longitude':1223,'latitude':856},concat_dim='time',parallel=True)

In [20]:
ds=ds.chunk({'time':7,'longitude':1223,'latitude':856})
ds

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 90)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2014-01-01 2014-01-02 ... 2014-03-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(90, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    history:      File initialised at 2015-12-16T15:53:48.785210\nInitialised...
    Conventions:  CF-1.6
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2014/01/A20140101.L2OC_...

In [21]:
print('Dataset size %.1f GB' % (ds.nbytes/1E9))

Dataset size 25.2 GB


In [22]:
import zarr
zarr_path = '/home/jovyan/shared-data/aqua.P1D.aust.K_490.2014.zarr'

In [23]:
compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
encoding = {vname: {'compressor': compressor} for vname in ds.variables}

ds.to_zarr(zarr_path,mode='w',encoding=encoding)

In [24]:
ds = xr.open_zarr(zarr_path)
ds

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 90)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2014-01-01 2014-01-02 ... 2014-03-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(90, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    Conventions:  CF-1.6
    history:      File initialised at 2015-12-16T15:53:48.785210\nInitialised...
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2014/01/A20140101.L2OC_...